In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

'''
    Author:	 Matthew Ogden and Graham West
    Created:	01 Sep 2020
Description:	This code is a modified version of code written by Graham West at https://github.com/gtw2i/GA-Galaxy. 
                The pupose of this code is to genetrically modify the SPAM orbital parameters for galactic models.
'''

# Add parent directory
from sys import path as sysPath
from os import listdir
sysPath.append('../')

# Add custom modules
sysPath.append('../Machine_Score/')

from Support_Code import general_module as gm
from Support_Code import info_module as im
from Genetic_Algorithm import main_Genetic_Algorithm as ga
import main_SIMR as simr

# General modules
from os import path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# Print testing
gm.test()
im.test()
ga.test()
simr.test()

def test():
    print("SM: Hi!  You're in Matthew's main code for all things simulation.")

# For testing and developement from outside. 
new_func = None
def set_new_func( new_func_ptr ):
    global new_func
    new_func = new_func_ptr
# End set new function
    

GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
GA: Hi!  You're in Matthew's main code for all things genetric algorithm.
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [2]:
tDir = '../targetDir'
print('Before: \'%s\' \nAfter: \'%s\'' % (tDir,gm.validPath(tDir) ))
tDir = gm.validPath(tDir)
rInfo = None

tInfo = im.target_info_class( targetDir = tDir, printAll=True)
if tInfo.status == False:
    print("WARNING")
else:
    print("Target Good!")
    im.tabprint("Grabbing first model.")
    rInfo = tInfo.getRunInfo()
    #print(rInfo)
    if type(rInfo) == type(None):
        print("BAD RUN!!!!")
    else:
        print("RUN GOOOD!!!")
        im.tabprint('rInfo status: %s' % (rInfo.status) )

Before: '../targetDir' 
After: '/home/mbo2d/587722984435351614/'
IM: target_info_class.__init__:
	 - targetDir:  /home/mbo2d/587722984435351614/
	 - arg.targetDir:  None
IM: Target.initTargetDir():
	 - targetDir: /home/mbo2d/587722984435351614/
	 - targetDir: (True) /home/mbo2d/587722984435351614/
	 - infoDir: (True) /home/mbo2d/587722984435351614/information/
	 - baseInfoLoc: (True) /home/mbo2d/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /home/mbo2d/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /home/mbo2d/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /home/mbo2d/587722984435351614/plots/
IM: Target: Opening target info json
Target Good!
	 - Grabbing first model.
RUN GOOOD!!!
	 - rInfo status: True


# TO-DO List
- Ask about boundary files
- Ask about Maxes and Mins Graham's Code uses from Zoo Merger Files. 
- Ask about the trailing values on the Galaxy Zoo Merger Models files and is it ok to cut them like Graham does? 

In [3]:
zScores, mData = tInfo.getOrbParam()
print(zScores)
print(mData)

tParam = mData[0,:]  # target orbital parameters

[0.94594595 0.94117647 0.91891892 ... 0.         0.         0.        ]
[[-9.93853 -4.5805   3.27377 ...  0.       0.       0.     ]
 [-9.93853 -4.5805  20.84411 ...  1.       0.       0.     ]
 [-9.93853 -4.5805  18.86315 ...  0.       0.       0.     ]
 ...
 [-9.93853 -4.5805  18.17628 ...  1.       0.       0.     ]
 [-9.93853 -4.5805  -7.327   ...  0.       0.       0.     ]
 [-9.93853 -4.5805   0.86396 ...  0.       0.       0.     ]]


In [4]:

def ReadAndCleanupData( filePath, thresh ):
	
	print("Cleaning target file...")
	
	# read data into np array
	df = pd.read_csv( filePath, sep=',|\t', engine='python', header=None )
	data1 = df.values
	
	# remove unranked models
	ind = 0
	while( not math.isnan(data1[ind,-1]) ):
		ind += 1
	# end
	data2 = data1[0:ind,:]
	nModel = ind + 1
	
	# include human score and SPAM params
	cols = list(range(4,18)) + [ 1 ]
	data2 = data2[:,cols]
	
	# ignore bad zoo models
	data2 = data2[data2[:,-1]>=thresh,:]
	nModel = data2.shape[0]
	
	data2 = np.array( data2, dtype=np.float32 )
	
#	data2[0,2]  = 10
#	data2[0,5]  = -10
#	data2[0,10] = 20
#	data2[0,11] = 20
#	data2[0,12] = 100
#	data2[0,13] = 100
	
	data3 = deepcopy(data2)
	
	psi = []
	for i in range(nModel):
		psi_p = 1
		psi_s = 1
		
		if( data2[i,2] < 0 ):
			data2[i,2]  =  -1 * data2[i,2]
			data2[i,5]  =  -1 * data2[i,5]
			data2[i,10] = 180 + data2[i,10]
			data2[i,11] = 180 + data2[i,11]
		# end
		data2[i,10] %= 360
		data2[i,11] %= 360
		if( data2[i,10] > 180 ):
			data2[i,10] = data2[i,10] - 180
			data2[i,12] = -1 * data2[i,12]
		# end
		if( data2[i,11] > 180 ):
			data2[i,11] = data2[i,11] - 180
			data2[i,13] = -1 * data2[i,13]
		# end
		data2[i,12] %= 360
		data2[i,13] %= 360
		if( data2[i,12] > 180 ):
			data2[i,12] = data2[i,12] - 360
		# end
		if( data2[i,13] > 180 ):
			data2[i,13] = data2[i,13] - 360
		# end
		
		if( data2[i,12] > 90 ):
			data2[i,12] = data2[i,12] - 180
			psi_p = -1
		elif( data2[i,12] < -90 ):
			data2[i,12] = data2[i,12] + 180
			psi_p = -1
		# end
		if( data2[i,13] > 90 ):
			data2[i,13] = data2[i,13] - 180
			psi_s = -1
		elif( data2[i,13] < -90 ):
			data2[i,13] = data2[i,13] + 180
			psi_s = -1
		# end
		psi.append( [psi_p,psi_s] )
	# end
	psi = np.array(psi)
	
	# energy
	G = 1
	r = ( data2[:,0]**2 + data2[:,1]**2 + data2[:,2]**2 )**0.5
	U = -G*data2[:,6]*data2[:,7]/r
	v = ( data2[:,3]**2 + data2[:,4]**2 + data2[:,5]**2 )**0.5
	K = 0.5*data2[:,7]*v**2
#	c = np.log(1-K/U)
	c = (K+U)/(K-U)
	
	# convert p,s mass to ratio,total mass
	t = data2[:,6] + data2[:,7]
	f = data2[:,6] / t
	data2[:,6] = f
	data2[:,7] = t
	
	# spherical velocity
	phi   = ( np.arctan2( data2[:,4], data2[:,3] ) * 180.0 / np.pi ) % 360
	theta = ( np.arcsin( data2[:,5] / v ) * 180.0 / np.pi )
	
#	data2[:,2] = c
#	data2[:,3] = v
	data2[:,3] = c
#	data2[:,2] = K
#	data2[:,3] = U
	
	data2[:,4] = phi
	data2[:,5] = theta
	
#	"""
	Ap = np.abs(data2[:,8]**2*np.cos(data2[:,12]*np.pi/180.0))
	As = np.abs(data2[:,9]**2*np.cos(data2[:,13]*np.pi/180.0))
	
	data2[:,8] = Ap
	data2[:,9] = As
#	"""
	
	return data2, psi, nModel, len(cols)
	
# end





pFile = '../Genetic_Algorithm/587722984435351614_combined.txt'
print(path.exists(pFile))

zooThresh = 0.5

import math
from copy import deepcopy
data, psi, nModel, nCol = ReadAndCleanupData( pFile, zooThresh )
print("data: ", data.shape)
print("PSI: ", psi.shape)
print("nModel: ", nModel)
print("nCol: ", nCol)


True
Cleaning target file...
data:  (121, 15)
PSI:  (121, 2)
nModel:  121
nCol:  15


In [5]:
#####################
###   VARIABLES   ###
#####################


# target ID
zooBase = "587722984435351614"
targetInd = 0
nParam    = 14		# number of SPAM parameters
shrink   = 1.0		# fraction of max xLim

pReal   = data[targetInd,0:-1]
psiReal = psi[targetInd,:]
nBin = 200

# read max xLim and window bounds
XXX = np.loadtxt( "../Genetic_Algorithm/Bounds_587722984435351614.txt" )
#	mmm = XXX[:nParam,:]
bound = XXX[nParam:,:]

# get parameter stats
mins = np.min( data, axis=0 )[0:-1]
maxs = np.max( data, axis=0 )[0:-1]
mmm = [ mins, maxs ]
mmm = np.transpose(np.array(mmm))
#	print mmm
# modify max xLim
#	mmm[2,:]  = np.array([ -10.0,  10.0 ])
#	mmm[2,:]  = np.array([   0.0,   5.0 ])
mmm[2,:]  = np.array([   0.0,   mmm[2,1] ])  # sym
"""
mmm[3,:]  = np.array([  -0.9,   0.9 ])
mmm[4,:]  = np.array([   0.0, 360.0 ])
mmm[5,:]  = np.array([ -90.0,  90.0 ])
#	mmm[6,:]  = np.array([   0.2,   0.8 ])
#	mmm[7,:]  = np.array([  10.0,  70.0 ])
#	mmm[8,:]  = np.array([   0.5,  10.0 ])
#	mmm[9,:]  = np.array([   0.5,  10.0 ])
"""
mmm[10,:] = np.array([   0.0, 180.0 ])	# sym
mmm[11,:] = np.array([   0.0, 180.0 ])	# sym
mmm[12,:] = np.array([ -89.0,  89.0 ])	
mmm[13,:] = np.array([ -89.0,  89.0 ])	
# shrink xLim
xLim = np.zeros((nParam,2))
for i in range(nParam):
    xLim[i,0] = pReal[i] - shrink*(pReal[i] - mmm[i,0])
    xLim[i,1] = shrink*(mmm[i,1] - pReal[i]) + pReal[i]
# end

print("    min      real     max")
for i in range(nParam):
#		print xLim[i,0], pReal[i], xLim[i,1]#, pWidth[i]
    print( "{:8.2f} ".format(xLim[i,0]) + "{:8.2f} ".format(pReal[i]) + "{:8.2f} ".format(xLim[i,1]) )
# end
print(" ")

    min      real     max
   -9.94    -9.94    -9.94 
   -4.58    -4.58    -4.58 
    0.00     3.27    24.90 
   -0.39     0.29     0.98 
  146.41   258.49   283.50 
  -67.00   -23.28    76.38 
    0.30     0.49     0.80 
   18.18    48.02    65.98 
    2.38     9.13    18.57 
    3.34    17.65    33.76 
    0.00   129.69   180.00 
    0.00    36.81   180.00 
  -89.00   -41.78    89.00 
  -89.00    51.43    89.00 
 


In [6]:

def solve( param, nParam, nBin, bound, fileInd, psi ):
	
	p = deepcopy(param)
#	print p
	
	# add for psi
	if( psi[0] == -1 ):
		p[12] += 180.0
	# end
	if( psi[1] == -1 ):
		p[13] += 180.0
	# end
	
	# convert mass units
	f    = p[6]
	t    = p[7]
	p[6] = f*p[7]
	p[7] = (1.0-f)*p[7]
	
	G = 1
	c = p[3]
	# K/U
#	v = ( 2*G*p[6]*(np.exp(c)-1)/(p[0]**2+p[1]**2+p[2]**2)**0.5 )**0.5
	# K+U/K-U
	v = ( (1+c)/(1-c)*2*G*p[6]/(p[0]**2+p[1]**2+p[2]**2)**0.5 )**0.5
	
	"""
	c = p[2]
	v = p[3]
	z = ( ( 2*G*p[6]*(np.exp(c)-1)/p[3]**2 )**2 - p[0]**2 - p[1]**2 )**0.5
	if( np.isreal(z) ):
		p[2] = z
	else:
		p[2] = 0
	# end
	"""
	
	vx = v*math.cos(p[4]*np.pi/180.0)*math.cos(p[5]*np.pi/180.0)
	vy = v*math.sin(p[4]*np.pi/180.0)*math.cos(p[5]*np.pi/180.0)
	vz = v*                           math.sin(p[5]*np.pi/180.0)
	p[3] = vx
	p[4] = vy
	p[5] = vz
	
#	"""
	Ap = p[8]
	As = p[9]
	p[8] = np.abs(Ap/np.cos(p[12]*np.pi/180.0))**0.5
	p[9] = np.abs(As/np.cos(p[13]*np.pi/180.0))**0.5
#	"""
	
#	p[2] = 1000
	
	return p   # Matt's addition
	paramStr = ','.join( map(str, p[0:nParam]) ) + ',0'
	return paramStr
	# with flag
	#call("./basic_run_unpreturbed -o " + fileInd + " " + paramStr + " > SolveMetro.out", shell=True)
	
	# with flag
	RV   = np.loadtxt("basic_"     + fileInd + ".out")
	RV_u = np.loadtxt("basic_unp_" + fileInd + ".out")
#	print RV_u[0,:]
	
	dr = RV[-1,0:3]-RV_u[-1,0:3]
	for i in range(len(RV)/2+1):
		j = i + len(RV)/2
		RV_u[j,0:3] = RV_u[j,0:3] + dr
		RV_u[j,3:] = 0
	# end
	
	binC   = BinField( nBin, RV,   bound )
	binC_u = BinField( nBin, RV_u, bound )
	
	return binC, binC_u, RV, RV_u
	
# end

#solve( param, nParam, nBin, bound, fileInd, psi )
# T, V, RVt, RVv = solve( pReal, nParam, nBin, bound, "00", psiReal )
pCall = solve( pReal, nParam, nBin, bound, "00", psiReal )


### Graham's Code and Model Data
This is how Graham's GA code reads in the Galazy Zoo: Merger files and converts them to his GA coordinate system.
It also converts it back into the SPAM coordinate system before calling SPAM. 
NOTE:  His code converts back into one of the 8 true symmetries he identifies in his dissertation. 

In [7]:
print(" i :   Model 1  :   pReal  :  paramStr  : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, tParam[i], pReal[i], pCall[i], tParam[i]-pCall[i]) )

 i :   Model 1  :   pReal  :  paramStr  : diff
 0 : -9.94e+00 : -9.94e+00 : -9.94e+00 : -0.00
 1 : -4.58e+00 : -4.58e+00 : -4.58e+00 : +0.00
 2 : +3.27e+00 : +3.27e+00 : +3.27e+00 : -0.00
 3 : -5.00e-01 : +2.91e-01 : -5.00e-01 : -0.00
 4 : -2.46e+00 : +2.58e+02 : -2.46e+00 : +0.00
 5 : -1.08e+00 : -2.33e+01 : -1.08e+00 : -0.00
 6 : +2.33e+01 : +4.86e-01 : +2.33e+01 : +0.00
 7 : +2.47e+01 : +4.80e+01 : +2.47e+01 : +0.00
 8 : +3.50e+00 : +9.13e+00 : +3.50e+00 : -0.00
 9 : +5.32e+00 : +1.76e+01 : +5.32e+00 : +0.00
10 : +3.10e+02 : +1.30e+02 : +1.30e+02 : +180.00
11 : +3.68e+01 : +3.68e+01 : +3.68e+01 : +0.00
12 : +4.18e+01 : -4.18e+01 : -4.18e+01 : +83.57
13 : +5.14e+01 : +5.14e+01 : +5.14e+01 : +0.00


In [8]:

def convert_spam_to_ga( in_param ):
    
    if len( in_param.shape ) == 1:
        in_param = np.expand_dims(in_param, axis=0)
    
    out_param = deepcopy(in_param)
    psi = []
    
    for i in range( out_param.shape[0]):

        psi_p = 1
        psi_s = 1

        if( out_param[i,2] < 0 ):
            out_param[i,2]  =  -1 * out_param[i,2]
            out_param[i,5]  =  -1 * out_param[i,5]
            out_param[i,10] = 180 + out_param[i,10]
            out_param[i,11] = 180 + out_param[i,11]
        # end
        out_param[i,10] %= 360
        out_param[i,11] %= 360
        if( out_param[i,10] > 180 ):
            out_param[i,10] = out_param[i,10] - 180
            out_param[i,12] = -1 * out_param[i,12]
        # end
        if( out_param[i,11] > 180 ):
            out_param[i,11] = out_param[i,11] - 180
            out_param[i,13] = -1 * out_param[i,13]
        # end
        out_param[i,12] %= 360
        out_param[i,13] %= 360
        if( out_param[i,12] > 180 ):
            out_param[i,12] = out_param[i,12] - 360
        # end
        if( out_param[i,13] > 180 ):
            out_param[i,13] = out_param[i,13] - 360
        # end

        if( out_param[i,12] > 90 ):
            out_param[i,12] = out_param[i,12] - 180
            psi_p = -1
        elif( out_param[i,12] < -90 ):
            out_param[i,12] = out_param[i,12] + 180
            psi_p = -1
        # end
        if( out_param[i,13] > 90 ):
            out_param[i,13] = out_param[i,13] - 180
            psi_s = -1
        elif( out_param[i,13] < -90 ):
            out_param[i,13] = out_param[i,13] + 180
            psi_s = -1
            
        # end
        psi.append( [psi_p,psi_s] )
    # end for loop
    
    psi = np.array(psi)

    # energy
    G = 1
    r = ( out_param[:,0]**2 + out_param[:,1]**2 + out_param[:,2]**2 )**0.5
    U = -G*out_param[:,6]*out_param[:,7]/r
    v = ( out_param[:,3]**2 + out_param[:,4]**2 + out_param[:,5]**2 )**0.5
    K = 0.5*out_param[:,7]*v**2
    c = (K+U)/(K-U)

    # convert p,s mass to ratio,total mass
    t = out_param[:,6] + out_param[:,7]
    f = out_param[:,6] / t
    out_param[:,6] = f
    out_param[:,7] = t

    # spherical velocity
    phi   = ( np.arctan2( out_param[:,4], out_param[:,3] ) * 180.0 / np.pi ) % 360
    theta = ( np.arcsin( out_param[:,5] / v ) * 180.0 / np.pi )

    out_param[:,3] = c

    out_param[:,4] = phi
    out_param[:,5] = theta

    Ap = np.abs(out_param[:,8]**2*np.cos(out_param[:,12]*np.pi/180.0))
    As = np.abs(out_param[:,9]**2*np.cos(out_param[:,13]*np.pi/180.0))

    out_param[:,8] = Ap
    out_param[:,9] = As

    return out_param, psi

gaData, gaPsi= ga.convert_spam_to_ga(mData)
p2 = gaData[0,:]
p2_psi = gaPsi[0,:]

print(pReal.shape)
print(p2.shape)
print(p2_psi)

print(" i :    real   : converted :   target  : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, pReal[i], p2[i], tParam[i], pReal[i]-p2[i]) )

(14,)
(34,)
[1 1]
 i :    real   : converted :   target  : diff
 0 : -9.94e+00 : -9.94e+00 : -9.94e+00 : +0.00
 1 : -4.58e+00 : -4.58e+00 : -4.58e+00 : -0.00
 2 : +3.27e+00 : +3.27e+00 : +3.27e+00 : +0.00
 3 : +2.91e-01 : +2.91e-01 : -5.00e-01 : +0.00
 4 : +2.58e+02 : +2.58e+02 : -2.46e+00 : +0.00
 5 : -2.33e+01 : -2.33e+01 : -1.08e+00 : +0.00
 6 : +4.86e-01 : +4.86e-01 : +2.33e+01 : +0.00
 7 : +4.80e+01 : +4.80e+01 : +2.47e+01 : -0.00
 8 : +9.13e+00 : +9.13e+00 : +3.50e+00 : +0.00
 9 : +1.76e+01 : +1.76e+01 : +5.32e+00 : -0.00
10 : +1.30e+02 : +1.30e+02 : +3.10e+02 : -0.00
11 : +3.68e+01 : +3.68e+01 : +3.68e+01 : +0.00
12 : -4.18e+01 : -4.18e+01 : +4.18e+01 : +0.00
13 : +5.14e+01 : +5.14e+01 : +5.14e+01 : -0.00


In [9]:
def convert_ga_to_spam( in_param, psi ):
    
    # Expand in parameters if a single vector
    if len( in_param.shape ) == 1:
        in_param = np.expand_dims(in_param, axis=0)
    
    # Create seperate matrix for output parameters
    out_param = deepcopy(in_param)
    
    for i in range( out_param.shape[0]):
        
        # add for psi
        if( psi[i,0] == -1 ):
            out_param[i,12] += 180.0
        # end
        if( psi[i,1] == -1 ):
            out_param[i,13] += 180.0
        # end

        # convert mass units
        f    = out_param[i,6]
        t    = out_param[i,7]
        out_param[i,6] = f*out_param[i,7]
        out_param[i,7] = (1.0-f)*out_param[i,7]

        G = 1
        c = out_param[i,3]
        v = ( (1+c)/(1-c)*2*G*out_param[i,6]/(out_param[i,0]**2+out_param[i,1]**2+out_param[i,2]**2)**0.5 )**0.5    

        vx = v*math.cos(out_param[i,4]*np.pi/180.0)*math.cos(out_param[i,5]*np.pi/180.0)
        vy = v*math.sin(out_param[i,4]*np.pi/180.0)*math.cos(out_param[i,5]*np.pi/180.0)
        vz = v*math.sin(out_param[i,5]*np.pi/180.0)
        out_param[i,3] = vx
        out_param[i,4] = vy
        out_param[i,5] = vz
        
        Ap = out_param[i,8]
        As = out_param[i,9]
        out_param[i,8] = np.abs(Ap/np.cos(out_param[i,12]*np.pi/180.0))**0.5
        out_param[i,9] = np.abs(As/np.cos(out_param[i,13]*np.pi/180.0))**0.5
        
    return out_param

print(p2_psi)

spamData = ga.convert_ga_to_spam( gaData, gaPsi )
p3 = spamData[0,:]

#print(tParam)
print("tParam",tParam.shape)
print("p2", p2.shape)
print("p3", p3.shape)

print(" i :    target   :   p2   :     p3    : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, p2[i], p3[i], tParam[i], tParam[i]-p3[i]) )


[1 1]
tParam (34,)
p2 (34,)
p3 (34,)
 i :    target   :   p2   :     p3    : diff
 0 : -9.94e+00 : -9.94e+00 : -9.94e+00 : +0.00
 1 : -4.58e+00 : -4.58e+00 : -4.58e+00 : +0.00
 2 : +3.27e+00 : +3.27e+00 : +3.27e+00 : +0.00
 3 : +2.91e-01 : -5.00e-01 : -5.00e-01 : -0.00
 4 : +2.58e+02 : -2.46e+00 : -2.46e+00 : +0.00
 5 : -2.33e+01 : -1.08e+00 : -1.08e+00 : +0.00
 6 : +4.86e-01 : +2.33e+01 : +2.33e+01 : +0.00
 7 : +4.80e+01 : +2.47e+01 : +2.47e+01 : +0.00
 8 : +9.13e+00 : +3.50e+00 : +3.50e+00 : -0.00
 9 : +1.76e+01 : +5.32e+00 : +5.32e+00 : +0.00
10 : +1.30e+02 : +1.30e+02 : +3.10e+02 : +180.00
11 : +3.68e+01 : +3.68e+01 : +3.68e+01 : +0.00
12 : -4.18e+01 : -4.18e+01 : +4.18e+01 : +83.57
13 : +5.14e+01 : +5.14e+01 : +5.14e+01 : +0.00


## Comparison
Here we can see my code matches Graham's Code for the very first model.  This was not tested on any other model parameters. 

In [11]:

print(" i :   target  :   pCall   :     p3    : diff")
for i in range(pReal.shape[0]):
    print( "%2d : %+5.2e : %+5.2e : %+5.2e : %+5.2f" % (i, tParam[i], pCall[i], p3[i], pCall[i]-p3[i]) )


 i :   target  :   pCall   :     p3    : diff
 0 : -9.94e+00 : -9.94e+00 : -9.94e+00 : +0.00
 1 : -4.58e+00 : -4.58e+00 : -4.58e+00 : -0.00
 2 : +3.27e+00 : +3.27e+00 : +3.27e+00 : +0.00
 3 : -5.00e-01 : -5.00e-01 : -5.00e-01 : +0.00
 4 : -2.46e+00 : -2.46e+00 : -2.46e+00 : -0.00
 5 : -1.08e+00 : -1.08e+00 : -1.08e+00 : +0.00
 6 : +2.33e+01 : +2.33e+01 : +2.33e+01 : -0.00
 7 : +2.47e+01 : +2.47e+01 : +2.47e+01 : -0.00
 8 : +3.50e+00 : +3.50e+00 : +3.50e+00 : +0.00
 9 : +5.32e+00 : +5.32e+00 : +5.32e+00 : -0.00
10 : +3.10e+02 : +1.30e+02 : +1.30e+02 : -0.00
11 : +3.68e+01 : +3.68e+01 : +3.68e+01 : +0.00
12 : +4.18e+01 : -4.18e+01 : -4.18e+01 : +0.00
13 : +5.14e+01 : +5.14e+01 : +5.14e+01 : -0.00
